In [ ]:
import numpy as np
import cv2
import os
from skimage import color, exposure, feature
import matplotlib.pyplot as plt
from skimage import io
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from skimage import io
from skimage import exposure
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Path to the root folder of your dataset
root_folder = r"F:\Eye Disease\Data"

# List all subdirectories (assuming each subdirectory corresponds to a class)
class_folders = ["cataract", "diabetic_retinopathy", "normal", "glaucoma"]

features_list = []
labels_list = []

def extract_hog_features(image):
    # Compute the HOG features
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1), visualize=True)

    # Rescale histogram for better display
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    return fd, hog_image_rescaled

for class_name in class_folders:
    class_folder = os.path.join(root_folder, class_name)

    for image_filename in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_filename)

        # Load the image
        image = io.imread(image_path)

        # Convert the image to grayscale
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Resize image to the same size (e.g., 64x64)
        image_gray = cv2.resize(image_gray, (256, 256))

        # Extract HOG features and visualize
        hog_features, hog_image = extract_hog_features(image_gray)

        # Display the original image and the HOG features
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

        ax1.axis('off')
        ax1.imshow(image_gray, cmap=plt.cm.gray)
        ax1.set_title('Original Image')

        ax2.axis('off')
        ax2.imshow(hog_image, cmap=plt.cm.gray)
        ax2.set_title('HOG Features')

        plt.show()

        # Append HOG features to the features list
        features_list.append(hog_features)

        # Append the label to the labels list
        labels_list.append(class_name)

In [ ]:
features_array = np.array(features_list)
labels_array = np.array(labels_list)

# Use LabelEncoder to convert class names into numeric labels
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels_array)

print("\n*********\n")
print(len(features_array))

In [ ]:
df = pd.DataFrame(data=features_array, columns=[f'HOG_feature_{i}' for i in range(features_array.shape[1])])
df['Label'] = numeric_labels

In [ ]:
df.head()

In [ ]:
csv_filename = "Features.csv"
df.to_csv(csv_filename, index=False)